# Working with EMV Data in ISO 8583

EMV (Europay, Mastercard, Visa) chip card data is transmitted in Field 55 of ISO 8583 messages. This notebook covers parsing and building EMV data.

## Setup

In [1]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.emv import EMV_TAGS, build_emv_data, parse_emv_data
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.types import ISO8583Message

## Understanding EMV TLV Format

EMV data uses Tag-Length-Value (TLV) encoding:

```
| Tag (1-2 bytes) | Length (1-3 bytes) | Value (variable) |
```

### Tag Structure
- 1-byte tags: High nibble determines tag class and type
- 2-byte tags: First byte ends with `1F` (all low bits set)

In [2]:
# Common EMV tags
common_tags = {
    "9F26": "Application Cryptogram (AC)",
    "9F27": "Cryptogram Information Data (CID)",
    "9F10": "Issuer Application Data (IAD)",
    "9F37": "Unpredictable Number",
    "9F36": "Application Transaction Counter (ATC)",
    "95": "Terminal Verification Results (TVR)",
    "9A": "Transaction Date",
    "9C": "Transaction Type",
    "5F2A": "Transaction Currency Code",
    "82": "Application Interchange Profile (AIP)",
    "9F1A": "Terminal Country Code",
    "9F03": "Amount, Other (cashback)",
    "9F33": "Terminal Capabilities",
    "9F34": "Cardholder Verification Method (CVM) Results",
    "9F35": "Terminal Type",
    "9F09": "Application Version Number",
    "9F41": "Transaction Sequence Counter",
}

print("Common EMV Tags (Field 55):")
print("-" * 60)
for tag, description in common_tags.items():
    print(f"Tag {tag:6s}: {description}")

Common EMV Tags (Field 55):
------------------------------------------------------------
Tag 9F26  : Application Cryptogram (AC)
Tag 9F27  : Cryptogram Information Data (CID)
Tag 9F10  : Issuer Application Data (IAD)
Tag 9F37  : Unpredictable Number
Tag 9F36  : Application Transaction Counter (ATC)
Tag 95    : Terminal Verification Results (TVR)
Tag 9A    : Transaction Date
Tag 9C    : Transaction Type
Tag 5F2A  : Transaction Currency Code
Tag 82    : Application Interchange Profile (AIP)
Tag 9F1A  : Terminal Country Code
Tag 9F03  : Amount, Other (cashback)
Tag 9F33  : Terminal Capabilities
Tag 9F34  : Cardholder Verification Method (CVM) Results
Tag 9F35  : Terminal Type
Tag 9F09  : Application Version Number
Tag 9F41  : Transaction Sequence Counter


## Parsing EMV Data

Let's parse a sample EMV data string:

In [3]:
# Sample EMV data (hex-encoded TLV)
sample_emv = (
    "9F2608C2E572DA1A6E3B95"  # Application Cryptogram
    "9F270180"  # CID: ARQC
    "9F100706010A03A4B800"  # Issuer Application Data
    "9F370469AD7F93"  # Unpredictable Number
    "9F3602001F"  # ATC
    "950500000000"  # TVR (no errors)
    "9A03251215"  # Transaction Date: Dec 15, 2025
    "9C0100"  # Transaction Type: Purchase
    "5F2A020840"  # Currency: USD
    "820200"  # AIP
    "9F1A020840"  # Terminal Country: US
)

print(f"Raw EMV Data ({len(sample_emv)//2} bytes):")
print(sample_emv)
print()

# Parse the EMV data
parsed_emv = parse_emv_data(sample_emv)

print("Parsed EMV Tags:")
print("-" * 60)
for tag, value in parsed_emv.items():
    tag_name = EMV_TAGS.get(tag, "Unknown")
    print(f"Tag {tag:6s}: {value:30s} ({tag_name})")

Raw EMV Data (64 bytes):
9F2608C2E572DA1A6E3B959F2701809F100706010A03A4B8009F370469AD7F939F3602001F9505000000009A032512159C01005F2A0208408202009F1A020840

Parsed EMV Tags:
------------------------------------------------------------
Tag 9F26  : C2E572DA1A6E3B95               (Application Cryptogram)
Tag 9F27  : 80                             (Cryptogram Information Data)
Tag 9F10  : 06010A03A4B800                 (Issuer Application Data)
Tag 9F37  : 69AD7F93                       (Unpredictable Number)
Tag 9F36  : 001F                           (Application Transaction Counter (ATC))
Tag 95    : 000000009A                     (Terminal Verification Results (TVR))
Tag 03    : 12159C01005F2A0208408202009F1A020840 (Unknown)


## Understanding Key EMV Tags

### Application Cryptogram (Tag 9F26)

The cryptogram is an 8-byte value generated by the chip card using 3DES or AES. It proves card authenticity.

In [4]:
# Cryptogram types based on CID (9F27)
cid_types = {
    "00": "AAC (Application Authentication Cryptogram) - Decline",
    "40": "TC (Transaction Certificate) - Approved offline",
    "80": "ARQC (Authorization Request Cryptogram) - Go online",
}

cid = parsed_emv.get("9F27", "")
print(f"CID Value: {cid}")
print(f"Meaning: {cid_types.get(cid, 'Unknown')}")

CID Value: 80
Meaning: ARQC (Authorization Request Cryptogram) - Go online


### Terminal Verification Results (Tag 95)

TVR is a 5-byte bitmap indicating card/terminal verification status:

In [5]:
def explain_tvr(tvr_hex: str) -> None:
    """Explain Terminal Verification Results."""
    tvr_bytes = bytes.fromhex(tvr_hex)

    # Byte 1 checks
    byte1_flags = [
        (0x80, "Offline data authentication not performed"),
        (0x40, "SDA failed"),
        (0x20, "ICC data missing"),
        (0x10, "Card on terminal exception file"),
        (0x08, "DDA failed"),
        (0x04, "CDA failed"),
    ]

    # Byte 2 checks
    byte2_flags = [
        (0x80, "ICC and terminal have different app versions"),
        (0x40, "Expired application"),
        (0x20, "Application not yet effective"),
        (0x10, "Requested service not allowed"),
        (0x08, "New card"),
    ]

    print(f"TVR: {tvr_hex}")
    print(f"Binary: {' '.join(format(b, '08b') for b in tvr_bytes)}")
    print("\nFlags Set:")

    all_clear = True
    for mask, desc in byte1_flags:
        if tvr_bytes[0] & mask:
            print(f"  - {desc}")
            all_clear = False

    for mask, desc in byte2_flags:
        if tvr_bytes[1] & mask:
            print(f"  - {desc}")
            all_clear = False

    if all_clear:
        print("  (All checks passed - no flags set)")


tvr = parsed_emv.get("95", "0000000000")
explain_tvr(tvr)

TVR: 000000009A
Binary: 00000000 00000000 00000000 00000000 10011010

Flags Set:
  (All checks passed - no flags set)


### Cardholder Verification Method Results (Tag 9F34)

CVM Results indicate how the cardholder was verified:

In [6]:
cvm_methods = {
    "00": "No CVM performed",
    "01": "Plaintext PIN verified by ICC",
    "02": "Enciphered PIN verified online",
    "03": "Plaintext PIN by ICC + signature",
    "04": "Enciphered PIN by ICC",
    "05": "Enciphered PIN by ICC + signature",
    "1E": "Signature required",
    "1F": "No CVM required",
    "3F": "No CVM performed (no CVM required)",
}

cvm_conditions = {
    "00": "Always",
    "01": "If unattended cash",
    "02": "If not unattended cash and not cash purchase",
    "03": "If terminal supports CVM",
    "04": "If manual cash",
    "05": "If purchase with cashback",
    "06": "If transaction in app currency and under X value",
    "07": "If transaction in app currency and over X value",
    "08": "If transaction in app currency and under Y value",
    "09": "If transaction in app currency and over Y value",
}

print("CVM Methods (Tag 9F34, Byte 1):")
for code, desc in cvm_methods.items():
    print(f"  {code}: {desc}")

CVM Methods (Tag 9F34, Byte 1):
  00: No CVM performed
  01: Plaintext PIN verified by ICC
  02: Enciphered PIN verified online
  03: Plaintext PIN by ICC + signature
  04: Enciphered PIN by ICC
  05: Enciphered PIN by ICC + signature
  1E: Signature required
  1F: No CVM required
  3F: No CVM performed (no CVM required)


## Building EMV Data

Let's build EMV data from scratch:

In [7]:
# Build EMV data with specific tags
emv_tags = {
    "9F26": "AABBCCDD11223344",  # Cryptogram (8 bytes)
    "9F27": "80",  # CID: ARQC
    "9F10": "06010A03A4B800",  # IAD
    "9F37": "12345678",  # Unpredictable Number
    "9F36": "0001",  # ATC
    "95": "0000000000",  # TVR (clean)
    "9A": "251215",  # Date: Dec 15, 2025
    "9C": "00",  # Transaction type: Purchase
    "5F2A": "0840",  # Currency: USD
    "82": "1980",  # AIP
    "9F1A": "0840",  # Country: US
    "9F34": "1F0302",  # CVM Results: No CVM required
}

built_emv = build_emv_data(emv_tags)
print(f"Built EMV Data ({len(built_emv)//2} bytes):")
print(built_emv)

# Verify by parsing back
print("\nVerification - Parsed back:")
reparsed = parse_emv_data(built_emv)
for tag, value in reparsed.items():
    print(f"  {tag}: {value}")

Built EMV Data (72 bytes):
9F2608AABBCCDD112233449F2701809F100706010A03A4B8009F3704123456789F36020001950500000000009A032512159C01005F2A020840820219809F1A0208409F34031F0302

Verification - Parsed back:
  9F26: AABBCCDD11223344
  9F27: 80
  9F10: 06010A03A4B800
  9F37: 12345678
  9F36: 0001
  95: 0000000000
  9A: 251215
  9C: 00
  5F2A: 0840
  82: 1980
  9F1A: 0840
  9F34: 1F0302


## EMV in ISO 8583 Messages

Field 55 carries EMV data in authorization requests:

In [8]:
builder = ISO8583Builder()
parser = ISO8583Parser()

# Build a chip card authorization with EMV data
chip_auth = ISO8583Message(
    mti="0100",
    fields={
        0: "0100",
        2: "4111111111111111",
        3: "000000",
        4: "000000010000",
        11: "123456",
        14: "2612",
        22: "051",  # Chip card entry
        23: "001",
        35: "4111111111111111=26125010000000000000",
        41: "TERM0001",
        42: "MERCHANT123456 ",
        49: "840",
        55: built_emv,  # EMV data in Field 55
    },
)

raw = builder.build(chip_auth)
print("Chip Card Authorization:")
print(f"Total message length: {len(raw)} bytes")
print("\nRaw message (first 100 chars):")
print(raw[:100])

Chip Card Authorization:
Total message length: 284 bytes

Raw message (first 100 chars):
01007024060020C082001641111111111111110000000000000100001234562612051001374111111111111111=261250100


In [9]:
# Parse the message and extract EMV data
parsed = parser.parse(raw)

print("Parsed Message:")
print(f"MTI: {parsed.mti}")
print(f"POS Entry Mode (F22): {parsed.fields.get(22)}")
print("\nField 55 (EMV Data):")

emv_field = parsed.fields.get(55, "")
if emv_field:
    emv_parsed = parse_emv_data(emv_field)
    for tag, value in emv_parsed.items():
        tag_name = EMV_TAGS.get(tag, "Unknown")
        print(f"  Tag {tag}: {value} ({tag_name})")

Parsed Message:
MTI: 0100
POS Entry Mode (F22): 051

Field 55 (EMV Data):
  Tag 9F26: AABBCCDD11223344 (Application Cryptogram)
  Tag 9F27: 80 (Cryptogram Information Data)
  Tag 9F10: 06010A03A4B800 (Issuer Application Data)
  Tag 9F37: 12345678 (Unpredictable Number)
  Tag 9F36: 0001 (Application Transaction Counter (ATC))
  Tag 95: 0000000000 (Terminal Verification Results (TVR))
  Tag 9A: 251215 (Transaction Date)
  Tag 9C: 00 (Transaction Type)
  Tag 5F2A: 0840 (Transaction Currency Code)
  Tag 82: 1980 (Application Interchange Profile (AIP))
  Tag 9F1A: 0840 (Terminal Country Code)
  Tag 9F34: 1F0302 (Cardholder Verification Method (CVM) Results)


## Response EMV Data (Field 55 in Response)

The issuer response may include EMV data for the chip to verify:

In [10]:
# Common response EMV tags
response_emv_tags = {
    "91": "Issuer Authentication Data",
    "71": "Issuer Script Template 1 (before AC)",
    "72": "Issuer Script Template 2 (after AC)",
    "89": "Authorization Code",
    "8A": "Authorization Response Code",
}

print("Response EMV Tags:")
print("-" * 50)
for tag, desc in response_emv_tags.items():
    print(f"  Tag {tag}: {desc}")

# Build response with EMV data
response_emv = build_emv_data(
    {
        "8A": "3030",  # Authorization Response Code "00"
        "91": "1234567890ABCDEF01020304",  # Issuer Auth Data
    }
)

print(f"\nResponse EMV Data: {response_emv}")

Response EMV Tags:
--------------------------------------------------
  Tag 91: Issuer Authentication Data
  Tag 71: Issuer Script Template 1 (before AC)
  Tag 72: Issuer Script Template 2 (after AC)
  Tag 89: Authorization Code
  Tag 8A: Authorization Response Code

Response EMV Data: 8A023030910C1234567890ABCDEF01020304


## Contactless EMV (NFC)

Contactless transactions use similar EMV tags but with some additions:

In [11]:
# Contactless-specific considerations
print("Contactless EMV Considerations:")
print("-" * 50)
print("1. POS Entry Mode (F22) = '07' for contactless chip")
print("2. POS Entry Mode (F22) = '91' for contactless mag stripe")
print("3. Terminal Type (9F35) indicates contactless capability")
print("4. CVM may be 'No CVM' for low-value contactless")
print("5. Transaction Limit checks in TVR")

# Contactless chip transaction
contactless_auth = ISO8583Message(
    mti="0100",
    fields={
        0: "0100",
        2: "4111111111111111",
        3: "000000",
        4: "000000002500",  # $25 (under CVM limit)
        11: "789012",
        22: "071",  # Contactless chip, PIN capable
        41: "TERM0001",
        42: "MERCHANT123456 ",
        55: build_emv_data(
            {
                "9F26": "FEDCBA9876543210",
                "9F27": "80",
                "9F34": "1F0302",  # No CVM required
                "9F35": "22",  # Terminal type: Contactless
                "95": "0000000000",
            }
        ),
    },
)

print("\nContactless Transaction:")
print(f"POS Entry: {contactless_auth.fields[22]}")
print(f"Amount: ${int(contactless_auth.fields[4])/100:.2f}")

Contactless EMV Considerations:
--------------------------------------------------
1. POS Entry Mode (F22) = '07' for contactless chip
2. POS Entry Mode (F22) = '91' for contactless mag stripe
3. Terminal Type (9F35) indicates contactless capability
4. CVM may be 'No CVM' for low-value contactless
5. Transaction Limit checks in TVR

Contactless Transaction:
POS Entry: 071
Amount: $25.00


## EMV Debugging Tips

Common issues when working with EMV data:

In [12]:
print("EMV Debugging Checklist:")
print("=" * 50)
print("""
1. CRYPTOGRAM VALIDATION FAILURES
   - Check ATC (9F36) is incrementing
   - Verify Unpredictable Number (9F37) is random
   - Confirm IAD (9F10) matches issuer format

2. TVR ERRORS (Tag 95)
   - Bit flags indicate specific failures
   - Common: offline auth failed, expired app

3. CVM FAILURES (Tag 9F34)
   - PIN required but not entered
   - Signature required but terminal can't capture

4. ISSUER SCRIPT FAILURES
   - Tag 71/72 execution errors
   - PIN change/unblock scripts

5. FORMAT ERRORS
   - Tag length mismatches
   - Missing mandatory tags
   - Hex encoding issues (odd length strings)
""")

EMV Debugging Checklist:

1. CRYPTOGRAM VALIDATION FAILURES
   - Check ATC (9F36) is incrementing
   - Verify Unpredictable Number (9F37) is random
   - Confirm IAD (9F10) matches issuer format

2. TVR ERRORS (Tag 95)
   - Bit flags indicate specific failures
   - Common: offline auth failed, expired app

3. CVM FAILURES (Tag 9F34)
   - PIN required but not entered
   - Signature required but terminal can't capture

4. ISSUER SCRIPT FAILURES
   - Tag 71/72 execution errors
   - PIN change/unblock scripts

5. FORMAT ERRORS
   - Tag length mismatches
   - Missing mandatory tags
   - Hex encoding issues (odd length strings)



## Summary

Key points for EMV data handling:

1. **Field 55** carries EMV data in TLV format
2. **Tag 9F26** (Cryptogram) proves card authenticity
3. **Tag 9F27** (CID) indicates cryptogram type (ARQC/TC/AAC)
4. **Tag 95** (TVR) shows verification results
5. **Tag 9F34** (CVM Results) shows how cardholder was verified
6. Use `parse_emv_data()` and `build_emv_data()` for TLV handling